<a href="https://colab.research.google.com/github/Catarinamesk/Catarina_Amb/blob/main/solution_pyspark_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Solução do Exercício PySpark - Processando Dados**
Este notebook contém a solução completa e comentada do exercício proposto com PySpark, utilizando o arquivo `sales.csv`. Todas as etapas estão detalhadas para facilitar o entendimento.

In [ ]:

# 1. Configurar PySpark no Google Colab
# Execute esta célula para instalar as dependências necessárias.
!pip install pyspark


In [ ]:

# 2. Importar bibliotecas
from pyspark.sql import SparkSession


In [ ]:

# 3. Criar a SparkSession
# A SparkSession é o ponto de entrada para trabalhar com PySpark.
spark = SparkSession.builder.appName("Solução PySpark").getOrCreate()


In [ ]:

# 4. Ler o arquivo CSV em um DataFrame
# Substitua "sales.csv" pelo caminho correto do arquivo no Colab (faça o upload do arquivo).
df = spark.read.csv("sales.csv", header=True, inferSchema=True)

# Exibir os primeiros registros para entender o formato dos dados
df.show(5)


+-------------+----------+---------+-----------+--------+--------+------------+------------------+------------+------+-------------+
|TransactionID|CustomerID|ProductID|ProductName|Category|Quantity|PricePerUnit|        TotalPrice|PurchaseDate|Region|PaymentMethod|
+-------------+----------+---------+-----------+--------+--------+------------+------------------+------------+------+-------------+
|            1|      1185|      103|     others|  simple|       8|      750.01|           6000.08|  2025-01-07| South|  Credit Card|
|            2|      4819|      136|      force|    sell|       2|      515.56|           1031.12|  2025-01-06|  West|Bank Transfer|
|            3|      4297|      140|       deal|   serve|       4|       980.5|            3922.0|  2025-01-01| North|  Credit Card|
|            4|      4365|      144|      later| another|       6|      507.34|           3044.04|  2025-01-06| North|       PayPal|
|            5|      2408|      176|        end|    baby|       9|   

In [ ]:

# 5. Filtrar vendas com TotalPrice acima de 3000
# Aplicamos o filtro para selecionar apenas transações maiores que 3000 no total.
vendas_filtradas = df.filter(df["TotalPrice"] > 3000)

# Exibir as vendas filtradas para verificar
vendas_filtradas.show(5)


+-------------+----------+---------+-----------+--------+--------+------------+----------+------------+------+-------------+
|TransactionID|CustomerID|ProductID|ProductName|Category|Quantity|PricePerUnit|TotalPrice|PurchaseDate|Region|PaymentMethod|
+-------------+----------+---------+-----------+--------+--------+------------+----------+------------+------+-------------+
|            1|      1185|      103|     others|  simple|       8|      750.01|   6000.08|  2025-01-07| South|  Credit Card|
|            3|      4297|      140|       deal|   serve|       4|       980.5|    3922.0|  2025-01-01| North|  Credit Card|
|            4|      4365|      144|      later| another|       6|      507.34|   3044.04|  2025-01-06| North|       PayPal|
|            7|      4318|      189|    defense|    wide|       8|      764.99|   6119.92|  2025-01-19|  West|       PayPal|
|           17|      2815|      115|      adult|  single|       6|       765.0|    4590.0|  2025-01-01|  East|  Credit Card|


In [ ]:

# 6. Salvar os dados filtrados em formato Parquet
# O formato Parquet é otimizado para análise em escala.
vendas_filtradas.write.parquet("vendas_filtradas.parquet", mode="overwrite")


In [ ]:

# 7. Criar uma tabela temporária para consultas SQL
# Isso permite executar consultas SQL diretamente no DataFrame.
df.createOrReplaceTempView("sales")


In [ ]:

# 8. Executar uma consulta SQL
# Aqui somamos o TotalPrice por região para entender o desempenho por localização.
resultado = spark.sql("SELECT Region, SUM(TotalPrice) AS TotalRevenue FROM sales GROUP BY Region")

# Exibir o resultado da consulta
resultado.show()


+------+-----------------+
|Region|     TotalRevenue|
+------+-----------------+
| South|6448566.390000022|
|  East|6220672.829999992|
|  West|6253902.339999993|
| North|6134006.539999987|
+------+-----------------+



In [ ]:

# 9. Salvar o resultado em formato CSV para análise posterior
resultado.write.csv("resultado_por_regiao.csv", mode="overwrite", header=True)
